In [1]:
import sys
sys.path.insert(0, "../mysite/windhunt/management/commands/")

In [2]:
import _windfinder
import _screen_scraping

In [7]:
def GetDay(tree, xpath):
    day1 = _screen_scraping.GetText(tree, xpath)[0]
    day1 = (day1[day1.find(",")+2:])
    return day1;


def GetDates(tree, xpath_runtime, xpath_day, xpath_hour):
    year = str(datetime.today().year)[2:] 
    forecast_hour = _screen_scraping.GetText(tree, xpath_hour)

    day1 = GetDay(tree, xpath_day)

    #print(year + " " + day1+" "+ time)
    
    runtime = [datetime.strptime(year + " " + day1+" "+ time, '%y %b %d %H:%M') for hour in forecast_hour]
    forecast_date = [datetime.strptime(year + " " + day1+" "+ hour[:-1], '%y %b %d %H') for hour in forecast_hour]
    return runtime, forecast_date;


def GetAngle(tree, xpath_angle):
    wind_angle = _screen_scraping.GetText(tree, xpath_angle)
    wind_angle = [s.replace('\n', '').replace('°', '').strip() for s in wind_angle]
    return wind_angle

In [29]:
import _screen_scraping
from datetime import datetime, timedelta
import pandas as pd


def GetForecast():
    page = 'https://www.windfinder.com/forecast/lippesee_paderborn'
    tree = _screen_scraping.GetHtmlData(page)

    df = pd.DataFrame(columns = ["runtime", "forecast_time","average","max", "angle"])
    rows = ["3","5","6","8","9"]
    columns = ['1','2']
    
    xpath_runtime = '//*[@id="last-update"]'
    xpath_day = '//*[@id="sidebar-ad-scaffold"]/div[1]/section/div[3]/div[1]/div[1]/h4'
    
    # counter for forecast days
    day = 0
    for x,d in enumerate(rows):
        for y,column in enumerate(columns):
            xpath_wind_max = '//*[@id="sidebar-ad-scaffold"]/div[1]/section/div['+d+']/div['+column+']/div[2]/div/div[4]/div[2]/span[2]'
            wind_speed_max = _screen_scraping.GetList(tree, xpath_wind_max)

            xpath_wind_average = '//*[@id="sidebar-ad-scaffold"]/div[1]/section/div['+d+']/div['+column+']/div[2]/div/div[4]/div[1]/div[1]/span/span[1]'
            wind_speed_average = _screen_scraping.GetList(tree, xpath_wind_average)
            
            xpath_angle ='//*[@id="sidebar-ad-scaffold"]/div[1]/section/div['+d+']/div['+column+']/div[2]/div/div[3]/span[1]'
            wind_angle = GetAngle(tree, xpath_angle)
            
            xpath_hour = '//*[@id="sidebar-ad-scaffold"]/div[1]/section/div['+d+']/div['+column+']/div[2]/div/div[2]/div/span'
            xpath_runtime = '//*[@id="last-update"]'
            
            runtime, forecast_time = GetDates(tree, xpath_runtime, xpath_day, xpath_hour)
            print(day)
            for i,f in enumerate(runtime):
                  df.loc[len(df)] = [pd.to_datetime(runtime[i]), pd.to_datetime(forecast_time[i]+timedelta(days = day)), wind_speed_average[i], wind_speed_max[i], wind_angle[i]]
            day +=1
    return df


In [3]:
df = _windfinder.GetForecast()

0
1
2
3
4
5
6
7
8
9


In [5]:
print(df)

               runtime       forecast_time average max angle
0  2018-12-15 11:48:00 2018-12-15 01:00:00       4   8   100
1  2018-12-15 11:48:00 2018-12-15 04:00:00       5   8    94
2  2018-12-15 11:48:00 2018-12-15 07:00:00       6  12   109
3  2018-12-15 11:48:00 2018-12-15 10:00:00       8  16   123
4  2018-12-15 11:48:00 2018-12-15 13:00:00       9  14   130
5  2018-12-15 11:48:00 2018-12-15 16:00:00       8  17   122
6  2018-12-15 11:48:00 2018-12-15 19:00:00      12  24   128
7  2018-12-15 11:48:00 2018-12-15 22:00:00      13  28   136
8  2018-12-15 11:48:00 2018-12-16 01:00:00      13  26   142
9  2018-12-15 11:48:00 2018-12-16 04:00:00      13  26   143
10 2018-12-15 11:48:00 2018-12-16 07:00:00      12  25   147
11 2018-12-15 11:48:00 2018-12-16 10:00:00      11  22   161
12 2018-12-15 11:48:00 2018-12-16 13:00:00       7  15   181
13 2018-12-15 11:48:00 2018-12-16 16:00:00       6  17   227
14 2018-12-15 11:48:00 2018-12-16 19:00:00       6  13   221
15 2018-12-15 11:48:00 2